# Multi-class Classification using neural networks

#### We are using a Kaggle dataset of bank clients and try to predict the corresponding credit card cathegory considering some features such as income, educational level, etc.

https://www.kaggle.com/datasets/thedevastator/predicting-credit-card-customer-attrition-with-m

#### As we are having 4 credit card classes, we will have to implement a softmax function to classify and we are not using logistic regression.

### Let's go!

In [51]:
#We import the following libraries to treat data and to build the classification model.

import numpy as np
import pandas as pd
import tensorflow as tf

#And these are the classses we bring to generate train/test sets and to create the model's layers

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [40]:
#First we bring the data and build a pandas dataframe

credit_card = pd.read_csv("Creditcard_classification.csv", sep=',')
df_credit_card = pd.DataFrame(credit_card)
df_credit_card

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.999910
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.999940
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.999980
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.999870
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,772366833,Existing Customer,50,M,2,Graduate,Single,$40K - $60K,Blue,40,...,4003.0,1851,2152.0,0.703,15476,117,0.857,0.462,0.000191,0.999810
10123,710638233,Attrited Customer,41,M,2,Unknown,Divorced,$40K - $60K,Blue,25,...,4277.0,2186,2091.0,0.804,8764,69,0.683,0.511,0.995270,0.004729
10124,716506083,Attrited Customer,44,F,1,High School,Married,Less than $40K,Blue,36,...,5409.0,0,5409.0,0.819,10291,60,0.818,0.000,0.997880,0.002118
10125,717406983,Attrited Customer,30,M,2,Graduate,Unknown,$40K - $60K,Blue,36,...,5281.0,0,5281.0,0.535,8395,62,0.722,0.000,0.996710,0.003294


In [41]:
#Let's clean de dataset to keep the data we see relevant for our model.
df_credit_card = df_credit_card[["Customer_Age","Dependent_count","Education_Level","Marital_Status","Income_Category","Card_Category"]]

In [42]:
#Let's understand all unique values of certain columns
print(df_credit_card["Education_Level"].unique())
print(df_credit_card["Marital_Status"].unique())
print(df_credit_card["Income_Category"].unique())
print(df_credit_card["Card_Category"].unique())

['High School' 'Graduate' 'Uneducated' 'Unknown' 'College' 'Post-Graduate'
 'Doctorate']
['Married' 'Single' 'Unknown' 'Divorced']
['$60K - $80K' 'Less than $40K' '$80K - $120K' '$40K - $60K' '$120K +'
 'Unknown']
['Blue' 'Gold' 'Silver' 'Platinum']


In [43]:
#We replace cathegorical values with scores to be able to train with the data.

df_credit_card_cl = df_credit_card['Education_Level'].replace(df_credit_card["Education_Level"].unique(),[2,4,1,0,3,5,6], inplace=True)
df_credit_card_cl = df_credit_card["Marital_Status"].replace(df_credit_card["Marital_Status"].unique(),[1,2,0,2], inplace=True)
df_credit_card_cl = df_credit_card["Income_Category"].replace(df_credit_card["Income_Category"].unique(),[3,1,4,2,5,0], inplace=True)
df_credit_card_cl = df_credit_card["Card_Category"].replace(df_credit_card["Card_Category"].unique(),[0,1,2,3], inplace=True)

C:\Users\augus\AppData\Local\Temp\ipykernel_23444\1615988539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit_card_cl = df_credit_card['Education_Level'].replace(df_credit_card["Education_Level"].unique(),[2,4,1,0,3,5,6], inplace=True)
C:\Users\augus\AppData\Local\Temp\ipykernel_23444\1615988539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_credit_card_cl = df_credit_card["Marital_Status"].replace(df_credit_card["Marital_Status"].unique(),[1,2,0,2], inplace=True)
C:\Users\augus\AppData\Local\Temp\ipykernel_23444\1615988539.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [44]:
#Here we have our cleaned dataset

df_credit_card

,Customer_Age,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category
0,45,3,2,1,3,0
1,49,5,4,2,1,0
2,51,3,4,1,4,0
3,40,4,2,0,1,0
4,40,3,1,1,3,0
...,...,...,...,...,...,...
10122,50,2,4,2,2,0
10123,41,2,0,2,2,0
10124,44,1,2,1,1,0
10125,30,2,4,0,2,0


In [45]:
#We create numpy arrays to define X and Y

X = np.array(df_credit_card[["Customer_Age","Dependent_count","Education_Level","Marital_Status","Income_Category"]])

Y = np.array(df_credit_card["Card_Category"])

print(X.shape)
print(Y.shape)

(10127, 5)
(10127,)


In [46]:
#Then, we prepare train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1)

print(X_train.shape)
print(X_test.shape)

(9114, 5)
(1013, 5)


In [71]:
#We define our model and train

model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),  
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

model.fit(
    X_train,y_train,
    epochs=10
)

Epoch 1/10
285/285 [==============================] - 1s 2ms/step - loss: 0.8931 - sparse_categorical_accuracy: 0.9173
Epoch 2/10
285/285 [==============================] - 0s 2ms/step - loss: 0.8930 - sparse_categorical_accuracy: 0.9335
Epoch 3/10
285/285 [==============================] - 0s 2ms/step - loss: 0.8930 - sparse_categorical_accuracy: 0.9335
Epoch 4/10
285/285 [==============================] - 1s 2ms/step - loss: 0.8930 - sparse_categorical_accuracy: 0.9335
Epoch 5/10
285/285 [==============================] - 1s 2ms/step - loss: 0.8930 - sparse_categorical_accuracy: 0.9335
Epoch 6/10
285/285 [==============================] - 1s 2ms/step - loss: 0.8930 - sparse_categorical_accuracy: 0.9335
Epoch 7/10
285/285 [==============================] - 0s 2ms/step - loss: 0.8930 - sparse_categorical_accuracy: 0.9335
Epoch 8/10
285/285 [==============================] - 0s 2ms/step - loss: 0.8930 - sparse_categorical_accuracy: 0.9335
Epoch 9/10
285/285 [============================

In [78]:
#We sample some outputs

sm_preferred = tf.nn.softmax(model.predict(X_train))
#sm_preferred = model.predict(X_train)
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

285/285 [==============================] - 0s 2ms/step
two example output vectors:
 [[8.9125985e-01 8.4486732e-05 1.0804182e-01 6.1382930e-04]
 [9.0684819e-01 3.4374263e-04 9.1708355e-02 1.0996795e-03]]
largest value 0.9926764 smallest value 6.3783136e-07


In [79]:
for i in range(5):
    print( f"{sm_preferred[i]}, category: {np.argmax(sm_preferred[i])}")
    print( f"{y_train[i]}, category: {np.argmax(y_train[i])}")

[8.9125985e-01 8.4486732e-05 1.0804182e-01 6.1382930e-04], category: 0
0, category: 0
[9.0684819e-01 3.4374263e-04 9.1708355e-02 1.0996795e-03], category: 0
0, category: 0
[9.7272384e-01 3.2922686e-05 2.7137555e-02 1.0568094e-04], category: 0
0, category: 0
[0.86732924 0.00145675 0.12697004 0.00424396], category: 0
0, category: 0
[0.76989657 0.00095084 0.22422822 0.00492441], category: 0
0, category: 0


In [80]:
#We calculate our model's accuracy

print("Evaluate on train data")
results = model.evaluate(X_train, y_train)
print("train acc:",round(results[1]*100,2))

print("Evaluate on test data")
results = model.evaluate(X_test, y_test)
print("test acc:", round(results[1]*100,2))

Evaluate on train data
285/285 [==============================] - 1s 2ms/step - loss: 0.8930 - sparse_categorical_accuracy: 0.9335
train acc: 93.35
Evaluate on test data
32/32 [==============================] - 0s 2ms/step - loss: 1.0591 - sparse_categorical_accuracy: 0.9141
test acc: 91.41
